In [1]:
import os
import torch
import json
import numpy as np
import numpy.random as rd
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('default')

from pandas.plotting import scatter_matrix
from scipy.spatial import distance
from scipy.stats import percentileofscore

## Load the embeddings

In [2]:
path1 = "/neurospin/dico/agaudin/Runs/03_monkeys/Output/analysis_folders/pca/30/Run1/pca_embeddings.csv"

embeddings1 = pd.read_csv(path1, index_col=0)

print(embeddings1.head())

embeddings2 = -embeddings1
embeddings2.head()

                    dim1        dim2        dim3        dim4        dim5  \
Subject                                                                    
1-03HUBJO_t0  -53.579821  283.674235 -289.220584  -39.377640   31.373385   
1-08ANDTI_t0  304.421732  221.746751  212.028836   33.319015 -129.834469   
1-11LEBJO_t0  229.580833  287.437274   26.512003 -120.822792   20.726943   
1-15LEHMI_t0  111.156533    3.509716  -62.315849  -17.678537 -288.433520   
1-17COLMA_t0 -155.806756 -150.651878 -204.962518 -183.430474 -253.461319   

                    dim6        dim7        dim8        dim9       dim10  ...  \
Subject                                                                   ...   
1-03HUBJO_t0 -158.339627    9.670317  152.284108  -65.771396 -149.046171  ...   
1-08ANDTI_t0   21.031527  119.441280  -10.692840   71.951305  124.713433  ...   
1-11LEBJO_t0    5.467387  187.373954  122.830636  -23.879877  -63.080933  ...   
1-15LEHMI_t0    9.222002  -45.920334  106.299119  199.909948  

,dim1,dim2,dim3,dim4,dim5,dim6,dim7,dim8,dim9,dim10,...,dim21,dim22,dim23,dim24,dim25,dim26,dim27,dim28,dim29,dim30
Subject,,,,,,,,,,,,,,,,,,,,,
1-03HUBJO_t0,53.579821,-283.674235,289.220584,39.377640,-31.373385,158.339627,-9.670317,-152.284108,65.771396,149.046171,...,-87.509182,-53.117284,24.258174,48.823370,-3.300542,-7.509744,63.925171,72.504643,-102.564855,89.569466
1-08ANDTI_t0,-304.421732,-221.746751,-212.028836,-33.319015,129.834469,-21.031527,-119.441280,10.692840,-71.951305,-124.713433,...,22.659114,-77.277498,-63.190227,85.281956,116.815156,28.785544,19.209134,-141.514044,138.735023,57.747222
1-11LEBJO_t0,-229.580833,-287.437274,-26.512003,120.822792,-20.726943,-5.467387,-187.373954,-122.830636,23.879877,63.080933,...,50.180483,-72.835848,-171.984233,119.051368,50.734883,-130.860103,-42.828971,141.733466,123.745309,17.192685
1-15LEHMI_t0,-111.156533,-3.509716,62.315849,17.678537,288.433520,-9.222002,45.920334,-106.299119,-199.909948,-192.257399,...,37.666366,51.478220,109.851240,106.756577,60.081274,-94.729260,40.706943,17.842731,84.573856,24.888063
1-17COLMA_t0,155.806756,150.651878,204.962518,183.430474,253.461319,89.578872,43.501400,11.313596,-115.759039,-53.291902,...,63.945028,59.527924,44.080342,7.834308,-55.770208,86.571211,54.449158,69.077240,24.373003,-94.710483


In [3]:
embeddings1.loc['1-17COLMA_t0', :]

dim1    -155.806756
dim2    -150.651878
dim3    -204.962518
dim4    -183.430474
dim5    -253.461319
dim6     -89.578872
dim7     -43.501400
dim8     -11.313596
dim9     115.759039
dim10     53.291902
dim11   -199.851824
dim12    -38.700631
dim13     67.739853
dim14    -59.144676
dim15    179.745536
dim16     37.512961
dim17    -53.069641
dim18    110.252272
dim19    129.498434
dim20     66.243626
dim21    -63.945028
dim22    -59.527924
dim23    -44.080342
dim24     -7.834308
dim25     55.770208
dim26    -86.571211
dim27    -54.449158
dim28    -69.077240
dim29    -24.373003
dim30     94.710483
Name: 1-17COLMA_t0, dtype: float64

In [4]:
# get the subject list

# small check before
if embeddings1.index.all() != embeddings2.index.all():
    raise ValueError("The two spaces don't contain the same subjects")
else:
    list_subjects = embeddings1.index.astype('str')

list_subjects

Index(['1-03HUBJO_t0', '1-08ANDTI_t0', '1-11LEBJO_t0', '1-15LEHMI_t0',
       '1-17COLMA_t0', '1-18ROUCO_t0', '1-20FORHU_t0', '1-23CREES_t0',
       '1-40AYMJU_t0', '2-02FERMA_t0',
       ...
       'sub-inhibition20_ses-pretraining_T1w',
       'sub-inhibition21_ses-pretraining_T1w',
       'sub-inhibition22_ses-pretraining_T1w',
       'sub-inhibition23_ses-pretraining_T1w',
       'sub-inhibition24_ses-pretraining_T1w',
       'sub-inhibition25_ses-pretraining_T1w',
       'sub-inhibition26_ses-pretraining_T1w',
       'sub-inhibition28_ses-pretraining_T1w',
       'sub-inhibition29_ses-pretraining_T1w',
       'sub-inhibition31_ses-pretraining_T1w'],
      dtype='object', name='Subject', length=341)

## Compute the space distances

In [5]:
def get_distance_matrix(emb, verbose=False):
    # emb should not have the nn nor the min_dist columns    
    dist_mat = pd.DataFrame()

    for idx in emb.index:
        line = emb[emb.index == idx]
        if verbose:
            print(line)
        distances = emb.apply(distance.euclidean, axis=1, args=[np.array(line)])
        dist_mat[idx] = distances

    return dist_mat

In [6]:
dist_mat_1 = get_distance_matrix(embeddings1)
dist_mat_2 = get_distance_matrix(embeddings2)

In [7]:
dist_mat_1['1-03HUBJO_t0'].sort_values().index

Index(['1-03HUBJO_t0', 'e0071_t1_s03', 'nih_chp_01746_t1', 'nih_chp_05041_t1',
       'nih_chp_05167_t1', 'a0011_t1_s03', 'nih_chp_04282_t1',
       'nih_chp_05651_t1', 'nih_chp_05192_t1', 'nih_chp_04633_t1',
       ...
       'sub-inhibition20_ses-pretraining_T1w', '1-20FORHU_t0',
       'nih_chp_02741_t1', 'sub-inhibition25_ses-pretraining_T1w',
       'a0024_t1_s03', 'a0039_t1_s03', 'nih_chp_01161_t1', 'a0022_t1_s03',
       'nih_chp_04572_t1', 'nih_chp_00901_t1'],
      dtype='object', name='Subject', length=341)

In [23]:
def get_percentile_matrix(dist_mat, verbose=False):
    # the rankings for a given subject are stored in a column (not a line)
    list_subjects = dist_mat.index
    n_sj = len(list_subjects)
    ranking_mat = pd.DataFrame(np.zeros((n_sj, n_sj)), columns=list_subjects, 
                               index=list_subjects)
    for sj in list_subjects:
        distances = dist_mat[sj]
        neighbours = distances.sort_values().index
        for i,neighbour in enumerate(neighbours):
            ranking_mat.loc[neighbour, sj] = i
    
    return(ranking_mat*100/n_sj)

In [10]:
perc_mat_1 = get_percentile_matrix(dist_mat_1)
perc_mat_2 = get_percentile_matrix(dist_mat_2)
perc_mat_1

Subject,1-03HUBJO_t0,1-08ANDTI_t0,1-11LEBJO_t0,1-15LEHMI_t0,1-17COLMA_t0,1-18ROUCO_t0,1-20FORHU_t0,1-23CREES_t0,1-40AYMJU_t0,2-02FERMA_t0,...,sub-inhibition20_ses-pretraining_T1w,sub-inhibition21_ses-pretraining_T1w,sub-inhibition22_ses-pretraining_T1w,sub-inhibition23_ses-pretraining_T1w,sub-inhibition24_ses-pretraining_T1w,sub-inhibition25_ses-pretraining_T1w,sub-inhibition26_ses-pretraining_T1w,sub-inhibition28_ses-pretraining_T1w,sub-inhibition29_ses-pretraining_T1w,sub-inhibition31_ses-pretraining_T1w
Subject,,,,,,,,,,,,,,,,,,,,,
1-03HUBJO_t0,0.000000,93.255132,50.439883,94.134897,56.011730,36.363636,97.653959,43.401760,66.275660,86.803519,...,95.014663,68.914956,43.988270,89.736070,96.774194,99.120235,90.322581,70.674487,96.480938,68.035191
1-08ANDTI_t0,92.668622,0.000000,2.052786,19.354839,88.563050,96.774194,25.806452,60.997067,9.677419,80.351906,...,82.111437,42.228739,28.739003,83.870968,64.222874,49.560117,95.014663,91.788856,7.038123,89.149560
1-11LEBJO_t0,46.041056,3.519062,0.000000,49.560117,80.645161,83.284457,39.882698,34.604106,7.917889,64.222874,...,97.360704,49.266862,38.709677,67.155425,65.982405,87.096774,97.067449,95.307918,73.020528,72.140762
1-15LEHMI_t0,85.923754,23.753666,47.800587,0.000000,8.797654,64.222874,39.002933,31.964809,5.571848,31.671554,...,22.580645,69.208211,63.049853,46.627566,19.941349,18.768328,31.378299,64.516129,43.695015,85.337243
1-17COLMA_t0,71.260997,95.601173,90.615836,13.489736,0.000000,0.293255,74.486804,54.545455,65.395894,92.375367,...,26.686217,94.721408,95.894428,80.645161,82.697947,69.501466,20.821114,90.615836,87.683284,74.193548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sub-inhibition25_ses-pretraining_T1w,97.947214,49.853372,84.457478,18.768328,43.988270,66.862170,26.979472,22.287390,2.052786,42.815249,...,39.882698,74.780059,77.419355,82.111437,41.935484,0.000000,3.225806,85.630499,22.287390,93.255132
sub-inhibition26_ses-pretraining_T1w,93.255132,97.653959,98.826979,35.483871,10.557185,16.422287,50.733138,78.005865,72.727273,24.926686,...,43.401760,88.269795,90.322581,69.794721,77.126100,1.759531,0.000000,75.659824,68.914956,44.868035
sub-inhibition28_ses-pretraining_T1w,4.692082,12.903226,14.662757,1.759531,4.985337,4.398827,11.436950,7.624633,4.105572,6.451613,...,1.173021,4.105572,14.076246,1.173021,1.466276,7.331378,5.865103,0.000000,1.759531,3.225806


In [9]:
def get_distance(ranking_matrix_1, ranking_matrix_2, fct=np.sqrt, ponderation=None, verbose=False):
    ranking_matrix_1 = fct(ranking_matrix_1)
    ranking_matrix_2 = fct(ranking_matrix_2)
    compute_matrix = ranking_matrix_1 - ranking_matrix_2
    compute_matrix = np.abs(compute_matrix)
    return compute_matrix.values.mean()

In [12]:
get_distance(perc_mat_1, perc_mat_2)

0.0

In [72]:
# def functions

def custom_ln(x):
    return np.log(x+1)

def custom_identity(x):
    return x

def custom_ln_100(x):
    return np.log(x+1)/np.log(100)

In [20]:
def latent_space_distance(emb1, emb2, fct=np.sqrt, ponderation=None, verbose=False):
    dist_mat_1 = get_distance_matrix(emb1, verbose=verbose)
    dist_mat_2 = get_distance_matrix(emb2, verbose=verbose)

    perc_mat_1 = get_percentile_matrix(dist_mat_1, verbose=verbose)
    perc_mat_2 = get_percentile_matrix(dist_mat_2, verbose=verbose)

    return get_distance(perc_mat_1, perc_mat_2, fct=fct, ponderation=ponderation, verbose=verbose)

In [61]:
latent_space_distance(embeddings1, embeddings2)

0.0

## Apply the method to collected data

In [62]:
path1 = "/neurospin/dico/agaudin/Runs/03_monkeys/Output/analysis_folders/pca/30/Run1/pca_embeddings.csv"
path2 = "/neurospin/dico/agaudin/Runs/03_monkeys/Output/analysis_folders/pca/30/Run2/pca_embeddings.csv"

embeddings1 = pd.read_csv(path1, index_col=0)
embeddings2 = pd.read_csv(path2, index_col=0)

latent_space_distance(embeddings1, embeddings2)

0.12437126816237146

In [24]:
# for a good net
path1 = "/neurospin/dico/agaudin/Runs/03_monkeys/Output/analysis_folders/convnet/no_foldlabel_30/13-15-48/cingulate_ACCpatterns_embeddings/full_embeddings.csv"
path2 = "/neurospin/dico/agaudin/Runs/03_monkeys/Output/analysis_folders/convnet/no_foldlabel_30/16-39-35/cingulate_ACCpatterns_embeddings/full_embeddings.csv"

embeddings1 = pd.read_csv(path1, index_col=0)
embeddings2 = pd.read_csv(path2, index_col=0)

latent_space_distance(embeddings1, embeddings2)

1.2966030495127052

In [64]:
# for a bad net
path1 = "/neurospin/dico/agaudin/Runs/03_monkeys/Output/analysis_folders/densenet2/no_foldlabel_4/11-41-24/cingulate_ACCpatterns_embeddings/full_embeddings.csv"
path2 = "/neurospin/dico/agaudin/Runs/03_monkeys/Output/analysis_folders/densenet2/no_foldlabel_4/15-20-30/cingulate_ACCpatterns_embeddings/full_embeddings.csv"

embeddings1 = pd.read_csv(path1, index_col=0)
embeddings2 = pd.read_csv(path2, index_col=0)

latent_space_distance(embeddings1, embeddings2)

2.633784175344139

In [33]:
# heavily unoptimized: recompute the distance matrices each time

paths1 = ["/neurospin/dico/agaudin/Runs/03_monkeys/Output/analysis_folders/pca/30/Run1/pca_embeddings.csv",
          "/neurospin/dico/agaudin/Runs/03_monkeys/Output/analysis_folders/convnet/no_foldlabel_30/13-15-48/cingulate_ACCpatterns_embeddings/full_embeddings.csv",
          "/neurospin/dico/agaudin/Runs/03_monkeys/Output/analysis_folders/densenet2/no_foldlabel_4/11-41-24/cingulate_ACCpatterns_embeddings/full_embeddings.csv"]
paths2 = ["/neurospin/dico/agaudin/Runs/03_monkeys/Output/analysis_folders/pca/30/Run2/pca_embeddings.csv",
          "/neurospin/dico/agaudin/Runs/03_monkeys/Output/analysis_folders/convnet/no_foldlabel_30/16-39-35/cingulate_ACCpatterns_embeddings/full_embeddings.csv",
          "/neurospin/dico/agaudin/Runs/03_monkeys/Output/analysis_folders/densenet2/no_foldlabel_4/15-20-30/cingulate_ACCpatterns_embeddings/full_embeddings.csv"]

idx_names = ['same', 'pca', 'good_net', 'bad_net', 'random']

functions = [np.sqrt, custom_ln, custom_identity]
functions_names = ['sqrt', 'custom_ln', 'custom_identity']

n_rows = len(idx_names)
n_fct = len(functions) + 1

results_df = pd.DataFrame(np.zeros((n_rows, n_fct)), columns=['original_nn']+functions_names, 
                          index=idx_names)


for j,fct in enumerate(functions):
    print(functions_names[j])
    for i in range(len(paths1)):
        embs1 = pd.read_csv(paths1[i], index_col=0)
        embs2 = pd.read_csv(paths2[i], index_col=0)
        value = latent_space_distance(embs1, embs2, fct=fct)
        print(i+1,value)
        results_df[functions_names[j]][idx_names[i+1]] = value

# random embeddings
random_embs_1 = pd.DataFrame(np.random.normal(size=(341,30)))
random_embs_2 = pd.DataFrame(np.random.normal(size=(341,30)))

for j,fct in enumerate(functions):
    print(functions_names[j])
    # random spaces
    value = latent_space_distance(random_embs_1, random_embs_2, fct=fct)
    print('random', value)
    results_df[functions_names[j]]['random'] = value
    # almost identical spaces (just a test)
    value = latent_space_distance(random_embs_1, -random_embs_1, fct=fct)
    print('identical', value)
    results_df[functions_names[j]]['same'] = value

results_df

sqrt
1 0.12437126816237146
2 1.2966030495127052
3 2.6337841753441387
          original_nn      sqrt  custom_ln  custom_identity
same              0.0  0.000000        0.0              0.0
pca               0.0  0.124371        0.0              0.0
good_net          0.0  1.296603        0.0              0.0
bad_net           0.0  2.633784        0.0              0.0
random            0.0  0.000000        0.0              0.0
custom_ln
1 0.045858203556882714
2 0.4606483775542624
3 0.9099148227208752
          original_nn      sqrt  custom_ln  custom_identity
same              0.0  0.000000   0.000000              0.0
pca               0.0  0.124371   0.045858              0.0
good_net          0.0  1.296603   0.460648              0.0
bad_net           0.0  2.633784   0.909915              0.0
random            0.0  0.000000   0.000000              0.0
custom_identity
1 1.5006170838913047
2 16.07434876698349
3 33.08522955351785
          original_nn      sqrt  custom_ln  custom_identity

,original_nn,sqrt,custom_ln,custom_identity
same,0.0,0.000000,0.000000,0.000000
pca,0.0,0.124371,0.045858,1.500617
good_net,0.0,1.296603,0.460648,16.074349
bad_net,0.0,2.633784,0.909915,33.085230
random,0.0,2.660778,0.919808,33.368853


In [37]:
random_embs_1 = pd.DataFrame(np.random.normal(size=(600,30)))
random_embs_2 = pd.DataFrame(np.random.normal(size=(600,30)))

random = []

for j,fct in enumerate(functions):
    print(functions_names[j])
    value = latent_space_distance(random_embs_1, random_embs_2, fct=fct)
    print(value)
    random.append(value)

random

sqrt
2.6390250830268314
custom_ln
0.9157099816607436
custom_identity
33.01176574074075


[2.6390250830268314, 0.9157099816607436, 33.01176574074075]

In [80]:
main_path = "/neurospin/dico/agaudin/Runs/03_monkeys/Output/analysis_folders/convnet/foldlabel_30"
save_path = "/neurospin/dico/agaudin/Runs/03_monkeys/Output/analysis_folders/#performance_comparisons/nn/convnet_foldlabel_30"

models = [model for model in os.listdir(main_path)
          if os.path.isdir(os.path.join(main_path, model))]
models.sort()

print(models)

functions = [custom_identity, np.sqrt, custom_ln, custom_ln_100]
functions_names = ['identity', 'sqrt', 'ln', 'ln_100']

for i in range(len(models)-1):
    for j in range(i+1, len(models)):
        model_path1 = os.path.join(main_path, models[i])
        model_path2 = os.path.join(main_path, models[j])
        
        embs1 = pd.read_csv(model_path1+"/cingulate_ACCpatterns_embeddings/full_embeddings.csv", index_col=0)
        embs2 = pd.read_csv(model_path2+"/cingulate_ACCpatterns_embeddings/full_embeddings.csv", index_col=0)

        values = {}

        dist_mat1 = get_distance_matrix(embs1)
        dist_mat2 = get_distance_matrix(embs2)

        perc_mat1 = get_percentile_matrix(dist_mat1)
        perc_mat2 = get_percentile_matrix(dist_mat2)

        for k,fct in enumerate(functions):
            values[functions_names[k]] = get_distance(perc_mat1, perc_mat2, fct=fct)
            print(models[i], models[j], functions_names[k], values[functions_names[k]])
        
        with open(os.path.join(save_path, models[i]+'_'+models[j]+"_nne.json"), 'w') as file:
            json.dump(values, file)        

['09-02-52', '11-19-12', '11-55-03', '13-51-11', '2022-07-05']
09-02-52 11-19-12 identity 15.291923162873152
09-02-52 11-19-12 sqrt 1.244928641498191
09-02-52 11-19-12 ln 0.44785660295783963
09-02-52 11-19-12 ln_100 0.09725082567426267
09-02-52 11-55-03 identity 14.099604656240126
09-02-52 11-55-03 sqrt 1.155143565170467
09-02-52 11-55-03 ln 0.4182005849961832
09-02-52 11-55-03 ln_100 0.09081110319627712
09-02-52 13-51-11 identity 14.429405398556602
09-02-52 13-51-11 sqrt 1.1788618468600471
09-02-52 13-51-11 ln 0.425508387294204
09-02-52 13-51-11 ln_100 0.09239797230271225
09-02-52 2022-07-05 identity 14.578815938869491
09-02-52 2022-07-05 sqrt 1.1925252312588084
09-02-52 2022-07-05 ln 0.4306774388230094
09-02-52 2022-07-05 ln_100 0.09352041758052912
11-19-12 11-55-03 identity 14.329188059231884
11-19-12 11-55-03 sqrt 1.1782414334567937
11-19-12 11-55-03 ln 0.42867084723094345
11-19-12 11-55-03 ln_100 0.09308469175259529
11-19-12 13-51-11 identity 14.597781019943573
11-19-12 13-51-11 s

In [84]:
fct = 'ln_100'

saved_path = "/neurospin/dico/agaudin/Runs/03_monkeys/Output/analysis_folders/#performance_comparisons/nn"
models = ['densenet2_foldlabel_30', 'densenet2_no_foldlabel_30', 'densenet2_foldlabel_4', 'densenet2_no_foldlabel_4',
'convnet_foldlabel_30', 'convnet_no_foldlabel_30', 'convnet_foldlabel_4', 'convnet_no_foldlabel_4']

mod_conds = [folder for folder in os.listdir(saved_path) if os.path.isdir(os.path.join(saved_path,folder))]

nne_mean = np.zeros((len(mod_conds)//2, 2))

for folder in mod_conds:
    j = 0 if 'convnet' in folder else 1
    if "no" in folder:
        i = 2
    else:
        i = 0
    if '4' in folder:
        i += 1
    
    func = []
    for file in os.listdir(os.path.join(saved_path,folder)):
        if 'nne' in file:

            with open(os.path.join(saved_path, folder, file), 'r') as file_txt:
                file_content = json.load(file_txt)
            func.append(file_content[fct])

    nne_mean[i,j] = np.mean(func)

df = pd.DataFrame(nne_mean, columns=['convnet', 'densenet2'], index=['foldlabel_30', 'foldlabel_4', 'no_foldlabel_30', 'no_foldlabel_4'])
df.to_csv(saved_path+f'/nne_{fct}.csv')
df

,convnet,densenet2
foldlabel_30,0.092403,0.137847
foldlabel_4,0.171030,0.175028
no_foldlabel_30,0.092500,0.096453
no_foldlabel_4,0.184234,0.168931


## Compare the results 

In [86]:
saved_path = "/neurospin/dico/agaudin/Runs/03_monkeys/Output/analysis_folders/#performance_comparisons/nn"

id_df = pd.read_csv(saved_path+'/nne_identity.csv', index_col=0)
sqrt_df = pd.read_csv(saved_path+'/nne_sqrt.csv', index_col=0)
ln_df = pd.read_csv(saved_path+'/nne_ln.csv', index_col=0)
ln_100_df = pd.read_csv(saved_path+'/nne_ln_100.csv', index_col=0)


nn_df = pd.read_csv(saved_path+'/mean_median_nn.csv', index_col=0)

print("id")
print(id_df)
print()

print("sqrt")
print(sqrt_df)
print()

print("ln")
print(ln_df)
print()

print("ln_100")
print(ln_100_df)
print()

print("nn")
print(nn_df)

The history saving thread hit an unexpected error (OperationalError('disk I/O error',)).History will not be written to the database.
id
                   convnet  densenet2
foldlabel_30     14.345135  22.484799
foldlabel_4      28.092495  28.596919
no_foldlabel_30  14.877203  15.825352
no_foldlabel_4   30.253163  27.949893

sqrt
                  convnet  densenet2
foldlabel_30     1.175234   1.806295
foldlabel_4      2.254452   2.301267
no_foldlabel_30  1.198850   1.264716
no_foldlabel_4   2.430580   2.234375

ln
                  convnet  densenet2
foldlabel_30     0.425531   0.634809
foldlabel_4      0.787621   0.806036
no_foldlabel_30  0.425977   0.444184
no_foldlabel_4   0.848430   0.777955

ln_100
                  convnet  densenet2
foldlabel_30     0.092403   0.137847
foldlabel_4      0.171030   0.175028
no_foldlabel_30  0.092500   0.096453
no_foldlabel_4   0.184234   0.168931

nn
                   convnet  densenet2
foldlabel_30      3.225806   8.929619
foldlabel_4      34.3

In [90]:
dfs = [id_df, sqrt_df, ln_df, ln_100_df, nn_df]

cond = 'foldlabel_4'

for df in dfs:
    ratio = df['convnet'][cond]/df['densenet2'][cond]
    print(ratio)

0.9823608942919595
0.979657041963818
0.9771532594690656
0.9771532594690657
0.9890109890109892


In [92]:
dfs = [id_df, sqrt_df, ln_df, ln_100_df, nn_df]

model = 'convnet'
cond1 = 'no_foldlabel_30'
cond2 = 'no_foldlabel_4'

for df in dfs:
    ratio = df[model][cond1]/df[model][cond2]
    print(ratio)

0.4917569487678245
0.4932364118837688
0.502076217279853
0.5020762172798531
0.05565217391304346


fct has almost no impact on the predictions: the absolute values might be different, but the ratio remains the same.